<a href="https://colab.research.google.com/github/Abbas-Muneer/Olympians/blob/DataScience/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAveragePooling2D,Input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive

In [18]:
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
%cd drive/MyDrive/model_training/


/content/drive/MyDrive/model_training


In [26]:
for i in os.listdir("content"):
  print(i)

resnet-model-23-02-21.h5
resnet_model_training


In [5]:
!ls raw-img/

 ACNE		     Basal_Cell_Carcinoma   Melanocytic_Nevi   Psoriasis
 Atopic_Dermatitis   Eczema		    Melanoma	      'Skin Cancer'


In [ ]:
for i in os.listdir("raw-img"):
  print(i,len(os.listdir("raw-img/" + i)))

ACNE 375
Skin Cancer 3543
Psoriasis 341
Basal_Cell_Carcinoma 1323
Melanocytic_Nevi 2020
Atopic_Dermatitis 481
Melanoma 1140
Eczema 18


In [ ]:
try:
  os.mkdir("train")
  os.mkdir("test")
except:
  pass  
for i in os.listdir("raw-img"):
  try:
    os.mkdir("train/" + i)
    os.mkdir("test/" + i)
  except:
    pass  
  for j in os.listdir("raw-img/" + i)[:1600]:
    os.rename("raw-img/"+i+"/"+j, "train/"+i+"/"+j)
  for j in os.listdir("raw-img/" + i)[:400]:
    os.rename("raw-img/"+i+"/"+j, "test/"+i+"/"+j)

In [ ]:
for i in os.listdir("train"):
  print(i,len(os.listdir("train/" + i)))

ACNE 1600
Psoriasis 1600
Basal_Cell_Carcinoma 1600
Melanocytic_Nevi 1600
Atopic_Dermatitis 1600
Melanoma 1600
Eczema 1600
Skin_Cancer 1600


In [ ]:
def img_Data(dir_path,target_size,batch,class_lst,preporcssing):
  if preporcssing:
    gen_object = ImageDataGenerator(preprocessing_function=preporcssing)
  else:
    gen_object = ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,
                                        target_size=target_size,
                                        batch_size=batch,
                                        class_mode='sparse',
                                        classes = class_lst,
                                        shuffle=True))

In [ ]:
train_data_gen = img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen = img_Data("test",(224,224),500,os.listdir("test"),preprocess_input)

Found 12799 images belonging to 8 classes.
Found 3200 images belonging to 8 classes.


In [ ]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2( input_shape=(224,224,3),
                                                             alpha=1.0,
                                                             include_top=False,
                                                             weights='imagenet',
                                                             input_tensor=None,
                                                             pooling=None,
                                                             classes=1000,
                                                             classifier_activation='softmax')
                                        
   


In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(8,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
              

In [ ]:
elst = callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck = callbacks.ModelCheckpoint(' .mdl_wt.hdf5',save_best_only=True,monitor='val_loss',mode= 'min')

In [ ]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=10)

In [20]:
#retrying model training with resnet model instead of mobileNetV2 model
!pip install tensorflow keras-tuner opencv-python matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import numpy as np
from matplotlib import pyplot as plt

In [22]:
DATA_TRAINING_PATH = 'train'
BATCH_SIZE = 32
IMG_HEIGHT, IMG_WIDTH = 224, 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH, 3)

DATA_TESTING_PATH = 'test'
BATCH_SIZE = 32
IMG_HEIGHT, IMG_WIDTH = 224, 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH, 3)

In [23]:
train_data = tf.keras.utils.image_dataset_from_directory(
    DATA_TRAINING_PATH,
    label_mode='categorical',
    color_mode='rgb',
    validation_split=0.0001,
    subset="training",
    seed=42,
    shuffle=True,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
    )

val_data = tf.keras.utils.image_dataset_from_directory(
    DATA_TESTING_PATH,
    label_mode='categorical',
    color_mode='rgb',
    validation_split=0.9999,
    subset="validation",
    seed=42,
    shuffle=False,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
    )

Found 12799 files belonging to 8 classes.
Using 12798 files for training.
Found 3200 files belonging to 8 classes.
Using 3199 files for validation.


In [24]:
class_names = train_data.class_names

# optimize
train_data = train_data.prefetch(tf.data.AUTOTUNE)
val_data = val_data.prefetch(tf.data.AUTOTUNE)

In [25]:
from tensorflow import keras
from keras.applications import ResNet50V2, VGG19, InceptionV3

In [26]:
base_model = ResNet50V2(
    include_top=False, # Exclude ImageNet classifier at the top
    weights='imagenet',
    input_shape=IMG_SIZE
    )



# Freeze the base_model
base_model.trainable = False

94668760/94668760 [==============================] - 1s 0us/step


In [28]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomTranslation, RandomContrast

In [27]:
data_augmentation = keras.Sequential(
    [RandomFlip('horizontal'), 
     RandomRotation(factor=(-0.025, 0.025)),
     RandomTranslation(height_factor=0.1, width_factor=0.1),
     RandomContrast(factor=0.1)
     ])



# Create new model on top
inputs = keras.Input(shape=IMG_SIZE)
x = data_augmentation(inputs)

x = keras.applications.resnet_v2.preprocess_input(x)
x = base_model(x, training=False)

In [3]:
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Flatten, Dense

In [ ]:
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)  # Regularize with dropout

outputs = Dense(len(class_names), activation="softmax", name="pred")(x)

resnet_model = keras.Model(inputs, outputs)



resnet_model.summary()

In [2]:
from tensorflow.keras.metrics import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

In [18]:
metrics = [CategoricalAccuracy(), Precision(), Recall()]

resnet_model.compile(
    loss = CategoricalCrossentropy(),
    optimizer = Adam(learning_rate=0.001),
    metrics = metrics
    )

es = EarlyStopping(patience=6, monitor='val_loss', mode='min', restore_best_weights=True)



resnet_history = resnet_model.fit(
      train_data, 
      epochs=25, 
      validation_data=val_data,
      callbacks=[es],
      verbose=1
      )



resnet_model.save('resnet-model-23-02-21.h5')



result = resnet_model.evaluate(val_data)
result




Epoch 1/25


400/400 [==============================] - 3572s 9s/step - loss: 1.0890 - categorical_accuracy: 0.6018 - precision: 0.6684 - recall: 0.5177 - val_loss: 0.9819 - val_categorical_accuracy: 0.6118 - val_precision: 0.6681 - val_recall: 0.5192
Epoch 2/25
400/400 [==============================] - 221s 549ms/step - loss: 0.8595 - categorical_accuracy: 0.6702 - precision: 0.7213 - recall: 0.6067 - val_loss: 0.9612 - val_categorical_accuracy: 0.6274 - val_precision: 0.6761 - val_recall: 0.5671
Epoch 3/25
400/400 [==============================] - 224s 557ms/step - loss: 0.7844 - categorical_accuracy: 0.6893 - precision: 0.7363 - recall: 0.6341 - val_loss: 0.9753 - val_categorical_accuracy: 0.6327 - val_precision: 0.6726 - val_recall: 0.5824
Epoch 4/25
400/400 [==============================] - 217s 540ms/step - loss: 0.7552 - categorical_accuracy: 0.7011 - precision: 0.7480 - recall: 0.6502 - val_loss: 0.9680 - val_categorical_accuracy: 0.6286 - val_precision: 0.6703 - val_recall: 0.5789
Epoch

[0.929154634475708, 0.6330103278160095, 0.6731724739074707, 0.5929977893829346]

In [37]:
!ls content/resnet_model_training


abbas1.jpg  ACNE.png  Atomic.jpg  psoriasis.jpg


In [ ]:
infection_CLASSES = {x:y for x,y in enumerate(class_names)}
MODEL_DIR = 'My Drive/model_training/content/resnet-model-23-02-21.h5'
MODEL = keras.models.load_model('content/resnet-model-23-02-21.h5')

def predict_image_class(img_path:str) -> str:
  img = keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)  # Create batch axis

  predictions = MODEL.predict(img_array)

  return infection_CLASSES[predictions.argmax(axis=1)[0]]



TEST_IMG_DIR = "content/resnet_model_training/psoriasis.jpg"
print(f"Prediction - {predict_image_class(TEST_IMG_DIR)}")



from google.colab import files
files.download('content/resnet-model-23-02-21.h5')